In [60]:
import numpy as np
import pandas as pd
import os

%matplotlib inline

#user specify instrument and contract month

product=''
month=''

while True:
    inst=int(raw_input('Please input the number of the product you want to see the seasonals: 1) Brent 2) Gasoil 3) HO 4) RBOB 5) WTI: '))
    
    if inst==1:
        product='Brent'
        break
    elif inst==2:
        product='Gasoil'
        break   
    elif inst==3:
        product='HO'
        break      
    elif inst==4:
        product='RBOB'
        break       
    elif inst==5:
        product='WTI'
        break       
    else:
        print 'You did not input a correct number. Please try again.'

while True:
    Mth=int(raw_input('Please input the month of the contract which you wish to evaluate in numbers from 1-12: '))
    
    if Mth==1:
        month='Jan'
        break
    elif Mth==2:
        month='Feb'
        break   
    elif Mth==3:
        month='Mar'
        break   
    elif Mth==4:
        month='Apr'
        break   
    elif Mth==5:
        month='May'
        break          
    elif Mth==6:
        month='Jun'
        break   
    elif Mth==7:
        month='Jul'
        break   
    elif Mth==8:
        month='Aug'
        break   
    elif Mth==9:
        month='Sep'
        break   
    elif Mth==10:
        month='Oct'
        break   
    elif Mth==11:
        month='Nov'
        break   
    elif Mth==12:
        month='Dec'
        break 
    else:
        print 'You did not input a correct number. Please try again.'
        

Please input the number of the product you want to see the seasonals: 1) Brent 2) Gasoil 3) HO 4) RBOB 5) WTI: 2
Please input the month of the contract which you wish to evaluate in numbers from 1-12: 3


In [61]:
cd Quandl

C:\Users\boonsin.teo\Documents\Quandl


In [62]:
cd $product

C:\Users\boonsin.teo\Documents\Quandl\Gasoil


In [63]:
cd $month

C:\Users\boonsin.teo\Documents\Quandl\Gasoil\Mar


In [64]:

count=0
path = os.getcwd()

#indicate evaluation dates

start_mth=raw_input('Please input start mth of seasonal period in MM format: ')
start_day=raw_input('Please input start day of seasonal period in DD format: ')

i=int(raw_input('Please input 1 if seasonal period start on same year as contract year and 0 if not: '))
            
end_mth=raw_input('Please input end mth of seasonal period in MM format: ')
end_day=raw_input('Please input end day of seasonal period in DD format: ')

j=int(raw_input('Please input 1 if seasonal period ends on same year as contract year and 0 if not: '))

pos=1

while True:

    a=raw_input("Please input 'y' if position is long and 'n' if position is short: ").lower()
    
    if a=='y':
        break
    if a=='n':
        pos=pos*(-1)
        break
    else:
        print "Please input either 'y' or 'n' for position"

print 'PnL will be calculated basis 1kb lot size for standardization purposes.'

listing = os.listdir(path)
for infile in listing:
    temp=pd.read_excel(infile) #read each file into panda dataframe
    
    contract_yr_str=infile[-8:-4] # get the contract year
    
    seasonal_year=int(contract_yr_str)-1 # get the seasonal year to create the index - prev year
    seasonal_year=str(seasonal_year) # convert to string
    
    start_date=str(int(seasonal_year)+i)+'-'+start_mth+'-'+start_day # create start date
    end_date=str(int(seasonal_year)+j)+'-'+end_mth+'-'+end_day # create end date
    
    dates=pd.date_range(start_date, end_date) # range of dates for date axis
    dates=dates.strftime('%Y-%m-%d') #make sure no left out dates
    sample_data=np.arange(0,len(dates)) #just sample numbers to create dataframe
    
    dates_dataframe=pd.DataFrame({'Date':dates,'data':sample_data}) # create dates dataframe
    dates_dataframe=dates_dataframe.set_index('Date') # set index as Date to facilitate joining
    
    temp=temp.set_index('Date') # set index as Date to facilitate joining
    
    temp=dates_dataframe.join(temp) # join dataframes together on Date
   

    temp.reset_index(inplace=True)
    
    temp=temp[temp['Date']>=start_date] # filter for start date

    temp=temp[temp['Date']<=end_date] # filter for end date
 

    temp=temp.set_index('Date') # creating new index as Date
    
    
    temp=temp.reset_index() # putting date back out into col
    temp['Date']=pd.to_datetime(temp['Date']).dt.strftime('%m/%d')
    temp=temp.set_index('Date')
    temp=pd.DataFrame(temp['Settle'],columns=['Settle']) 

    #combine into seasonal index table
    
    if count==0:
        Seasonal_Combi=temp
        seasonal_year=str(int(seasonal_year)+1)
        Seasonal_Combi.columns=[seasonal_year]
        
        count+=1
        
    else:
        
        seasonal_year=str(int(seasonal_year)+1)
        Seasonal_Combi[seasonal_year]=temp['Settle']

#remove columns that are all NA
Seasonal_Combi.dropna(axis=1,how='all',inplace=True)

#remove rows that are all NA
Seasonal_Combi.dropna(axis=0,how='all',inplace=True)

#fillna values with values of forward value
Seasonal_Combi.fillna(method='bfill',inplace=True)
#fillna values with values of back value
Seasonal_Combi.fillna(method='ffill',inplace=True)

#Items to Create
#PnL - use last value vs first value

z=1
factor=1000
if product =='RBOB':
    z=42
elif product=='HO':
    z=42
elif product=='Gasoil':
    factor=100
else:
    z=1
    factor=1000

pnl_series=(Seasonal_Combi.iloc[-1]-Seasonal_Combi.iloc[0])*z*pos*factor    

# Win %
Win_Percent=float(pnl_series[pnl_series>0].count())/float(pnl_series.count())*100
    
#Largest Drawdown - use max and min
max_series=Seasonal_Combi.max()
min_series=Seasonal_Combi.min()
if pos==1:
    max_drawdown=(min_series-Seasonal_Combi.iloc[0])*pos*z*factor
else:
    max_drawdown=(max_series-Seasonal_Combi.iloc[0])*pos*z*factor
    

#Plot Equity Curve - use plot below for all columns
#% return - calculate using last value vs first value - similar to pnl calc
percent_return=((Seasonal_Combi.iloc[-1]-Seasonal_Combi.iloc[0])/(Seasonal_Combi.iloc[0]))



"""  
#plot on line chart
ax=Seasonal_Combi.plot.line(x=Seasonal_Combi.index, y='SI_avg_rolling',figsize=(20,5),xticks=[x for x in range(0,250,10)],title=month+" "+product)
ax.set_xlabel('Date')
ax.set_ylabel('Seasonal Index')
ax.legend(loc=2)
"""

Please input start mth of seasonal period in MM format: 12
Please input start day of seasonal period in DD format: 23
Please input 1 if seasonal period start on same year as contract year and 0 if not: 0
Please input end mth of seasonal period in MM format: 02
Please input end day of seasonal period in DD format: 26
Please input 1 if seasonal period ends on same year as contract year and 0 if not: 1
Please input 'y' if position is long and 'n' if position is short: y
PnL will be calculated basis 1kb lot size for standardization purposes.


'  \n#plot on line chart\nax=Seasonal_Combi.plot.line(x=Seasonal_Combi.index, y=\'SI_avg_rolling\',figsize=(20,5),xticks=[x for x in range(0,250,10)],title=month+" "+product)\nax.set_xlabel(\'Date\')\nax.set_ylabel(\'Seasonal Index\')\nax.legend(loc=2)\n'

In [65]:
cd ..

C:\Users\boonsin.teo\Documents\Quandl\Gasoil


In [66]:
cd ..

C:\Users\boonsin.teo\Documents\Quandl


In [67]:
cd ..

C:\Users\boonsin.teo\Documents


In [68]:
Seasonal_Combi.head()

,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
Date,,,,,,,,,,,,,,,,,,,,,
12/23,169.0,205.25,160.25,100.25,199.75,229.0,167.25,244.00,252.00,380.50,...,810.50,436.00,624.5,791.75,907.0,924.00,942.25,564.00,352.75,491.5
12/24,169.0,208.25,159.75,99.25,198.75,229.0,167.25,245.25,254.25,372.00,...,810.50,421.25,620.5,786.50,917.5,924.00,941.00,557.25,355.75,507.5
12/25,169.0,208.00,155.25,100.25,203.00,229.0,175.75,248.50,256.00,351.75,...,823.50,419.25,641.5,785.00,917.5,924.00,941.00,553.25,351.25,507.5
12/26,169.0,208.00,155.25,100.25,203.00,229.0,175.75,248.50,256.00,351.75,...,823.50,419.25,641.5,785.00,917.5,939.75,947.25,553.25,351.25,507.5
12/27,169.0,208.00,155.25,100.25,203.00,229.0,175.75,248.50,256.00,351.75,...,827.75,423.75,641.5,785.00,917.5,935.00,953.00,549.25,351.25,507.5


In [69]:
pnl_series

1996      400.0
1997    -3950.0
1998    -2825.0
1999        0.0
2000     2750.0
2001     -500.0
2002     -425.0
2003     6875.0
2004     1875.0
2005     7150.0
2006     2275.0
2007      800.0
2008     8825.0
2009    -3575.0
2010      425.0
2011    12950.0
2012    12375.0
2013     3275.0
2014    -1575.0
2015     2175.0
2016    -2750.0
2017      300.0
dtype: float64

In [70]:
percent_return

1996    0.023669
1997   -0.192448
1998   -0.176287
1999    0.000000
2000    0.137672
2001   -0.021834
2002   -0.025411
2003    0.281762
2004    0.074405
2005    0.187911
2006    0.043919
2007    0.015002
2008    0.108883
2009   -0.081995
2010    0.006805
2011    0.163562
2012    0.136439
2013    0.035444
2014   -0.016715
2015    0.038564
2016   -0.077959
2017    0.006104
dtype: float64

In [71]:
max_drawdown

1996    -1925.0
1997    -3950.0
1998    -2825.0
1999     -900.0
2000    -1000.0
2001    -1550.0
2002    -1075.0
2003     -800.0
2004     -800.0
2005    -2875.0
2006     -750.0
2007    -6475.0
2008    -4650.0
2009    -7200.0
2010    -5000.0
2011    -1950.0
2012        0.0
2013        0.0
2014    -4625.0
2015   -10325.0
2016    -9775.0
2017     -900.0
dtype: float64

In [72]:
print 'The win percentage is %.2f percent' %Win_Percent

The win percentage is 63.64 percent


In [73]:
pwd

u'C:\\Users\\boonsin.teo\\Documents'